In [4]:
# import pyaudio
import wave
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
from keras.models import load_model
from keras.optimizers import rmsprop

Using TensorFlow backend.


In [ ]:
"""
01 = neutral, -
02 = calm, 
03 = happy, -
04 = sad, -
05 = angry, -
06 = fearful, -
07 = disgust, - 
08 = surprised -
"""

## Prediction

In [2]:
def predict(file_path):
    model_path = './saved_models/ser_weights2.h5'
    model = load_model(model_path)

    X, sample_rate = librosa.load(file_path, res_type='kaiser_fast',duration=3,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)

    opt = rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    labelled_class_path = './label_classes.npy'
    lb = LabelEncoder()
    lb.classes_ = np.load(labelled_class_path, allow_pickle=True)
    features = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    features_df= pd.DataFrame(data=features)
    features_stacked = features_df.stack().to_frame().T
    features_expanded= np.expand_dims(features_stacked, axis=2)
    predictions = model.predict(features_expanded, batch_size=512, verbose=1)
    predictions_mod = predictions.argmax(axis=1)
    preds_flat = predictions_mod.astype(int).flatten()
    predictions_array = (lb.inverse_transform((preds_flat)))
    
    return predictions_array[0]

In [6]:
file_path='./Audio_Speech_Actors_01-24/Actor_09/03-01-03-02-01-02-09.wav'
predict(file_path)

1/1 [==============================] - 0s 46ms/step


'happy'